In [19]:
import os
import re
import pandas as pd
import xml.etree.ElementTree as ET
import fitz  # PyMuPDF
from collections import defaultdict

In [20]:
# --- Konfigurasi Path ---
# Disesuaikan dengan struktur folder baru yang memiliki subfolder PDF dan XML
# --- Konfigurasi Path ---
BASE_PATH = r'D:\Jupyter File\make-data-count-finding-data-references'
TEST_FOLDER = os.path.join(BASE_PATH, 'test')
TEST_XML_FOLDER = os.path.join(TEST_FOLDER, 'XML')
TEST_PDF_FOLDER = os.path.join(TEST_FOLDER, 'PDF')
SAMPLE_SUBMISSION_PATH = os.path.join(BASE_PATH, 'sample_submission.csv')
OUTPUT_FILE = 'submission.csv'

In [21]:
# --- Fungsi Helper untuk Ekstraksi Teks ---

def clean_text(text):
    """Membersihkan teks dari spasi berlebih dan karakter aneh."""
    text = re.sub(r'-\n', '', text)  # Menghapus tanda hubung di akhir baris
    text = re.sub(r'\n', ' ', text)   # Mengganti baris baru dengan spasi
    text = re.sub(r'\s+', ' ', text)  # Mengganti spasi ganda dengan spasi tunggal
    return text.strip()

def extract_text_from_pdf(pdf_path):
    """Membaca dan mengekstrak teks dari file PDF."""
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for page in doc:
            text += page.get_text()
        doc.close()
        return clean_text(text.lower())
    except Exception as e:
        print(f"Gagal membaca PDF {pdf_path}: {e}")
        return ""

def extract_text_from_xml(xml_path):
    """Membaca dan mengekstrak teks dari file XML."""
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()
        text_parts = [elem.text for elem in root.iter() if elem.text]
        text = ' '.join(text_parts)
        return clean_text(text.lower())
    except Exception as e:
        print(f"Gagal membaca XML {xml_path}: {e}")
        return ""

def get_article_text(article_id, xml_folder, pdf_folder):
    """
    Mendapatkan teks lengkap dari artikel dari subfolder yang benar.
    Memprioritaskan file XML, jika tidak ada, gunakan PDF.
    """
    xml_path = os.path.join(xml_folder, f"{article_id}.xml")
    pdf_path = os.path.join(pdf_folder, f"{article_id}.pdf")
    
    if os.path.exists(xml_path):
        return extract_text_from_xml(xml_path)
    elif os.path.exists(pdf_path):
        return extract_text_from_pdf(pdf_path)
    return ""

In [22]:
# --- Pola Regular Expression (Regex) yang Diperluas ---
DOI_REGEX = r'\b(10\.\d{4,9}/[-._;()/:A-Z0-9]+)\b'

ACCESSION_ID_REGEX = {
    'GEO': r'\b(GSE\d{4,})\b',
    'SRA': r'\b(SRP\d{6,}|ERP\d{6,}|DRP\d{6,})\b',
    'PDB': r'\b(PDB\s\w{4}|\d\w{3})\b',
    'ARRAY_EXPRESS': r'\b(E-\w{4}-\d+)\b',
    'DRYAD': r'\b(dryad\.\w+\.?\w+)\b',
    'FIGSHARE': r'\b(figshare\.\w+\.?\w+)\b',
    'ZENODO': r'\b(zenodo\.\d+)\b',
    'GENBANK': r'\b([A-Z]{1,2}\d{5,})\b'
}

In [23]:
# --- Logika Klasifikasi Berbasis Kata Kunci ---
SECONDARY_KEYWORDS = [
    'obtained from', 'reused', 'retrieved from', 'acquired from', 
    'downloaded from', 'accessed from', 'taken from', 'derived from',
    'diperoleh dari', 'digunakan kembali', 'diambil dari', 'diunduh dari', 'diakses dari'
]

PRIMARY_KEYWORDS = [
    'data are available', 'can be accessed from', 'available from', 'deposited in',
    'generated as part of this paper', 'data for this study', 'are provided',
    'data we used in this publication', 'data necessary for confirming',
    'data tersedia', 'dapat diakses dari', 'tersedia dari', 'disimpan di',
    'dihasilkan sebagai bagian dari makalah ini', 'data untuk penelitian ini'
]

def classify_citation(ds_id, full_text):
    """
    Mengklasifikasikan kutipan sebagai Primary atau Secondary berdasarkan kata kunci.
    """
    context_window = 250
    for match in re.finditer(re.escape(ds_id), full_text, re.IGNORECASE):
        start, end = match.span()
        context = full_text[max(0, start - context_window):end + context_window]
        for keyword in SECONDARY_KEYWORDS:
            if keyword in context:
                return 'Secondary'
        for keyword in PRIMARY_KEYWORDS:
            if keyword in context:
                return 'Primary'
    return 'Primary'

In [ ]:
# --- Logika Utama ---
def main():
    """Fungsi utama untuk menjalankan proses deteksi dan klasifikasi."""
    print("Memulai proses penambangan kutipan data...")
    
    # Memastikan folder-folder yang dibutuhkan ada
    if not os.path.isdir(TEST_FOLDER):
        print(f"Error: Folder utama '{TEST_FOLDER}' tidak ditemukan.")
        return

    # Mendapatkan daftar ID artikel dari kedua subfolder (PDF dan XML)
    pdf_files = os.listdir(TEST_PDF_FOLDER) if os.path.isdir(TEST_PDF_FOLDER) else []
    xml_files = os.listdir(TEST_XML_FOLDER) if os.path.isdir(TEST_XML_FOLDER) else []
    
    test_article_ids = sorted(list(set([fn.split('.')[0] for fn in pdf_files + xml_files])))
    
    if not test_article_ids:
        print(f"Error: Tidak ada file yang ditemukan di dalam subfolder '{TEST_PDF_FOLDER}' atau '{TEST_XML_FOLDER}'.")
        return
        
    print(f"Ditemukan {len(test_article_ids)} artikel unik di dalam folder test.")
    
    predictions = []
    
    for i, article_id in enumerate(test_article_ids):
        if (i + 1) % 50 == 0 or i == len(test_article_ids) - 1:
            print(f"Memproses artikel {i+1}/{len(test_article_ids)}: {article_id}")
            
        full_text = get_article_text(article_id, TEST_XML_FOLDER, TEST_PDF_FOLDER)
        if not full_text:
            continue
            
        found_datasets = set()

        # Cari DOI
        for match in re.finditer(DOI_REGEX, full_text, re.IGNORECASE):
            dataset_id = match.group(0).strip('.,')
            found_datasets.add(dataset_id)
            
        # Cari Accession IDs
        for pattern in ACCESSION_ID_REGEX.values():
            for match in re.finditer(pattern, full_text, re.IGNORECASE):
                found_datasets.add(match.group(0).strip('.,'))

        for ds_id in found_datasets:
            citation_type = classify_citation(ds_id, full_text)
            predictions.append({
                'article_id': article_id,
                'dataset_id': ds_id,
                'type': citation_type
            })

    # Buat file submission
    if not predictions:
        print("Tidak ada dataset yang ditemukan. Membuat file submission kosong.")
        submission_df = pd.DataFrame(columns=['row_id', 'article_id', 'dataset_id', 'type'])
    else:
        submission_df = pd.DataFrame(predictions).drop_duplicates()
        submission_df.reset_index(drop=True, inplace=True)
        submission_df['row_id'] = submission_df.index
        submission_df = submission_df[['row_id', 'article_id', 'dataset_id', 'type']]

    submission_df.to_csv(OUTPUT_FILE, index=False)
    
    print(f"\nProses selesai. File submission telah dibuat di: {OUTPUT_FILE}")
    print(f"Total prediksi yang dibuat: {len(submission_df)}")
    print("Contoh isi file submission:")
    print(submission_df.head())

if __name__ == '__main__':
    main()

Memulai proses penambangan kutipan data...
Ditemukan 1 artikel unik di dalam folder test.
Memproses artikel 1/1: 10
Tidak ada dataset yang ditemukan. Membuat file submission kosong.

Proses selesai. File submission telah dibuat di: submission.csv
Total prediksi yang dibuat: 0
Contoh isi file submission:
Empty DataFrame
Columns: [row_id, article_id, dataset_id, type]
Index: []
